<a href="https://colab.research.google.com/github/ollorin/collabs/blob/main/llm_daemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask flask-ngrok

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/process', methods=['POST'])
def process_text():
    # Extract data from the POST request
    data = request.get_json()
    prompt = data.get('prompt', '')
    variables = data.get('variables', {})

    # Process the prompt with the variables
    processed_text = process_prompt(prompt, variables)

    # Return the processed text
    return jsonify({'processed_text': processed_text})

def process_prompt(prompt, variables):
    # Your prompt processing logic here
    # This is a placeholder for your code
    for key, value in variables.items():
        prompt = prompt.replace(f"{{{key}}}", str(value))
    return prompt

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://1cdc-35-229-70-205.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Jul/2023 20:42:09] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jul/2023 20:42:27] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jul/2023 20:42:29] "POST /process HTTP/1.1" 200 -
